In [1]:
from TSCFLP_instance import TSCFLPinstance
from TSCFLP_greedy_decoder import greedy_decoder
import torch
import torch.nn as nn
import numpy as np

In [2]:
class Generator(nn.Module):
    def __init__(self,I,J,population):
        super().__init__()
        self.population = population
        self.rede = nn.Sequential(
            nn.Linear(I+J,200),
            nn.ReLU(),
            nn.Linear(200,300),
            nn.ReLU(),
            nn.Linear(300,300),
            nn.ReLU(),
            nn.Linear(300,(I+J)*population),
            nn.Sigmoid()
        )
    def forward(self,ruido):
        x = self.rede(ruido)
        x = torch.reshape(x,(self.population,150))
        return x
        
        

In [3]:
class evaluate_population():
    def __init__(self,decoder):
        self.is_inicialized = False 
        self.decoder = decoder
        self.best_fit = 0
        self.best_chromosome = 0
        self.mean_last_pop = 0
    

    def evaluate_pop(self,population):
        pop_individual_fit = np.array([],dtype=int)
        for index,chromosome in enumerate(population):
            fit = self.decoder.decode(chromosome,True)
            pop_individual_fit = np.append(pop_individual_fit,fit)
        
        if self.is_inicialized == False:
            self.best_fit = np.min(pop_individual_fit)
            pop_mean = np.mean(pop_individual_fit)
            self.mean_last_pop = pop_mean
            x = torch.tensor([-(self.best_fit-pop_mean)])
            loss = np.exp(torch.tanh(x))
            self.is_inicialized = True
        else:
            pop_mean = np.mean(pop_individual_fit)
            x = torch.tensor([-(self.best_fit-pop_mean)])
            loss = np.exp(torch.tanh(x))
            max_individual_fit = np.max(pop_individual_fit)
                          
            if max_individual_fit > self.best_fit:
                self.best_fit = max_individual_fit
                
                 
        return loss
            
       
        
        
        
        
        

In [4]:
model = Generator(50,100,200)

In [5]:
ruido = torch.rand((150))

In [6]:
test = model(ruido)

In [7]:
print(len(test))

200


In [8]:
test.shape

torch.Size([200, 150])

In [9]:
instancia = TSCFLPinstance(r"C:\Users\Felipe\Projetos\TSCFLP\Instances\PSC1-C1-50.txt")
decoder = greedy_decoder(instancia)
evaluator = evaluate_population(decoder)

In [10]:
loss = evaluator.evaluate_pop(test)

In [11]:
print(loss)

tensor([2.7183], dtype=torch.float64)


In [12]:
print(evaluator.best_fit)

850268


In [13]:
print(evaluator.mean_last_pop)

941422.695
